# Extract The Data

We've learned how to create a database, insert data and then update or delete it.

We've also seen how to view the content of a table using pandas.

But we've only seen how to display the entire contents of a single table. What if we only want a subset of the records? What if we want to extract a set based on records in more than one table?

It's time to learn about the SELECT statement.

First, let's connect to our database, empty both tables and create records for three flights.

We'll need to enable foreign keys again as this setting applies to the connection:

In [29]:
import sqlalchemy as sa
import pandas as pd
from collections import namedtuple
engine = sa.create_engine('sqlite:///flight.db')
connection = engine.connect()
connection.execute("PRAGMA foreign_keys=on")
connection.execute("DELETE FROM flights")

Flight = namedtuple('Flight', 'name country_code')

flights = [
    Flight(name='hab1', country_code='GB'),
    Flight(name='hab2', country_code='GB'),
    Flight(name='hab3', country_code='US'),
]

sql = """
    INSERT INTO flights(name, country_code)
    VALUES(?, ?)
"""

for flight in flights:
    connection.execute(sql, (flight.name, flight.country_code))
pd.read_sql('flights', connection)

,name,country_code,latitude,longitude
0,hab1,GB,0.0,0.0
1,hab2,GB,0.0,0.0
2,hab3,US,0.0,0.0


And then generate 20 randomised readings for each flight:

In [30]:
from random import uniform, randint

Reading = namedtuple('Reading', 'flight, ts, temp, pressure, humidity')

readings = [
    Reading(
        flight=flight.name,
        ts=f'2015-01-01 09:{str(i+1).zfill(2)}:00',
        temp=round(uniform(23, 27), 1),
        pressure=randint(1020, 1025),
        humidity=randint(30, 50))
    for flight in flights
    for i in range(20)
]

sql = """
INSERT INTO readings
    (flight, ts, temp, pressure, humidity)
VALUES
    (?, ?, ?, ?, ?)
"""

for reading in readings:
    values = (reading.flight, reading.ts, reading.temp, reading.pressure, reading.humidity)
    connection.execute(sql, values)